In [1]:
#! /usr/bin/env python
"""
Compute elevation statistics for the debris-covered areas in each latitude and longitude
"""

import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from scipy import ndimage
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib

import debrisglobal.globaldebris_input as debris_prms

In [2]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

print('All DC glaciers:', dc_shp.shape[0], 'All DC Area (km2):', dc_shp.DC_Area_v2.sum() / 1e6)

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)

rgino_str_list_subset = [x.split('-')[1] for x in dc_shp_subset.RGIId.values]

print('Subset DC glaciers:', dc_shp_subset.shape[0], 'Subset DC Area (km2):', dc_shp_subset.DC_Area_v2.sum() / 1e6)

dc_shp_subset

All DC glaciers: 5782 All DC Area (km2): 8405.006654
Subset DC glaciers: 914 Subset DC Area (km2): 7728.282367


,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Name,DC_Area,DC_BgnDate,DC_EndDate,DC_CTSmean,DC_Area_%,area_singl,DC_Area_v2,DC_Area__1,geometry
0,RGI60-05.00115,G307874E65600N,20000823,-9999999,-52.114567,65.598312,5,1,72.377,143,...,None,2068200,2013,2017,41.791113,2.86,35126,1100729,1.521,"MULTIPOLYGON (((-52.12018 65.57680, -52.11953 ..."
1,RGI60-05.00181,G307071E65710N,20000823,-9999999,-52.930226,65.711906,5,1,2.060,111,...,None,135900,2013,2017,36.231972,6.60,108966,108966,5.290,"POLYGON ((-52.93602 65.72019, -52.93406 65.720..."
2,RGI60-05.00184,G307047E65700N,20000814,-9999999,-52.954854,65.694883,5,1,2.807,241,...,None,342000,2013,2017,30.275185,12.18,109866,232227,8.273,"MULTIPOLYGON (((-52.97294 65.69817, -52.97228 ..."
3,RGI60-05.00258,G307406E65840N,20000823,-9999999,-52.591557,65.839391,5,1,4.232,378,...,None,358200,2013,2017,41.271976,8.46,36925,253069,5.980,"MULTIPOLYGON (((-52.63166 65.84086, -52.63100 ..."
4,RGI60-05.00270,G307462E65832N,20000823,-9999999,-52.538784,65.829037,5,1,5.630,351,...,None,360900,2013,2017,38.827046,6.41,204438,288195,5.119,"MULTIPOLYGON (((-52.55089 65.81989, -52.55024 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909,RGI60-05.20173,G338962E74764N,20010820,-9999999,-21.039994,74.762030,5,1,3.393,827,...,None,352800,2013,2017,76.662043,10.40,21617,183634,5.412,"MULTIPOLYGON (((-21.00784 74.75812, -21.00682 ..."
910,RGI60-05.20183,G337398E74984N,20010820,-9999999,-22.602830,74.981868,5,1,3.712,1223,...,None,494100,2013,2017,75.935375,13.31,27020,353063,9.511,"MULTIPOLYGON (((-22.57486 74.97749, -22.56553 ..."
911,RGI60-05.20198,G337984E74902N,20010820,-9999999,-21.951378,74.885092,5,1,45.809,923,...,None,2562300,2013,2017,91.047647,5.59,144112,2336088,5.100,"MULTIPOLYGON (((-21.89461 74.87198, -21.89464 ..."
912,RGI60-05.20231,G337801E75066N,20010820,-9999999,-22.179918,75.062760,5,1,4.590,873,...,None,337500,2013,2017,91.285011,7.35,36928,281017,6.122,"MULTIPOLYGON (((-22.14201 75.05643, -22.13680 ..."


In [3]:
# Select glaciers using RGI and find unique latlons
#  (Scherler DC shapefiles do not have same CenLat and CenLon for some reason)
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgino_str_list_subset)
main_glac_rgi_subset['CenLon_360'] = main_glac_rgi_subset['CenLon']
main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + main_glac_rgi_subset.loc[main_glac_rgi_subset['CenLon_360'] < 0, 'CenLon_360'])

# Load met data and find nearest latlon indices
ds = xr.open_dataset(debris_prms.metdata_fp + '../' + debris_prms.metdata_elev_fn)
#  argmin() finds the minimum distance between the glacier lat/lon and the GCM pixel
lat_nearidx = (np.abs(main_glac_rgi_subset['CenLat'].values[:,np.newaxis] - 
                      ds['latitude'][:].values).argmin(axis=1))
lon_nearidx = (np.abs(main_glac_rgi_subset['CenLon_360'].values[:,np.newaxis] - 
                      ds['longitude'][:].values).argmin(axis=1))
latlon_nearidx = list(zip(lat_nearidx, lon_nearidx))
latlon_nearidx_unique = sorted(list(set(latlon_nearidx)))
main_glac_rgi_subset['latlon_nearidx'] = latlon_nearidx
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_subset['latlon_unique_no'] = main_glac_rgi_subset['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_subset['latlon_unique_no'])), '\n\n')

# Delete me
latlon_nearidx_unique_v1 = latlon_nearidx_unique.copy()

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)

914 glaciers in region 5 are included in this model run: ['00115', '00181', '00184', '00258', '00270', '00275', '00318', '00319', '00323', '00334', '00335', '00336', '00337', '00342', '00400', '00421', '00445', '00446', '00451', '00459', '00460', '00522', '00530', '00789', '00790', '00793', '00794', '00800', '00801', '00817', '00839', '00859', '00879', '00904', '00938', '01224', '01355', '01383', '01403', '01441', '01452', '01456', '01706', '01780', '02398', '02521', '02536', '02651', '02918', '03019'] and more
This study is focusing on 914 glaciers in region [5]
unique lat/lons: 634 




In [4]:
# ===== LOAD GLACIERS WITH DATA =====
main_glac_rgi_subset['mb_fn'] = np.nan
mb_binned_fp = debris_prms.mb_binned_fp

regions_str = [str(x).zfill(2) for x in debris_prms.roi_rgidict[debris_prms.roi]]

mb_fns = []
mb_rgiids = []
for i in os.listdir(mb_binned_fp):
    if i.endswith('_mb_bins.csv') and i.split('_')[0].split('.')[0].zfill(2) in regions_str:
        mb_fns.append(i)
        rgiid_raw = i.split('_')[0]
        rgiid = 'RGI60-' + rgiid_raw.split('.')[0].zfill(2) + '.' + rgiid_raw.split('.')[1]
        mb_rgiids.append(rgiid)
mb_rgiids = sorted(mb_rgiids)
mb_fns = sorted(mb_fns)
mb_fn_df = pd.DataFrame(np.zeros((len(mb_fns),2)), columns=['RGIId', 'mb_fn'])
mb_fn_df['RGIId'] = mb_rgiids
mb_fn_df['mb_fn'] = mb_fns

# Find glaciers that are debris-covered
mb_dc_rgiid = [value for value in list(mb_fn_df.RGIId.values) if value in list(main_glac_rgi_subset.RGIId.values)]
mb_fn_df_dc = mb_fn_df[mb_fn_df['RGIId'].isin(mb_dc_rgiid)]
mb_fn_df_dc = mb_fn_df_dc.sort_values('RGIId')

print('Debris-covered glaciers:', mb_fn_df_dc.shape[0], '\n\n')

mb_fn_dict = dict(zip(mb_fn_df_dc['RGIId'].values, mb_fn_df_dc['mb_fn'].values))

main_glac_rgi_subset['mb_fn'] = main_glac_rgi_subset.RGIId.map(mb_fn_dict)

Debris-covered glaciers: 611 




In [5]:
# ===== SELECT GLACIERS WITH DATA ====
main_glac_rgi_wobs = main_glac_rgi_subset.dropna(subset=['mb_fn']).copy()
# print('subset wdata length:', main_glac_rgi_wobs.shape)
main_glac_rgi_wobs.reset_index(inplace=True, drop=True)

# Update the latlon unique pickle files
latlon_nearidx_unique = sorted(list(set(main_glac_rgi_wobs['latlon_nearidx'].values)))
latlon_unique_dict = dict(zip(latlon_nearidx_unique,np.arange(0,len(latlon_nearidx_unique))))
latlon_unique_dict_reversed = dict(zip(np.arange(0,len(latlon_nearidx_unique)),latlon_nearidx_unique))
main_glac_rgi_wobs['latlon_unique_no'] = main_glac_rgi_wobs['latlon_nearidx'].map(latlon_unique_dict)

print('unique lat/lons:', len(np.unique(main_glac_rgi_wobs['latlon_unique_no'])), '\n\n')

lat_list = np.array([ds.latitude[x[0]].values for x in latlon_nearidx_unique])
lon_list = np.array([ds.longitude[x[1]].values for x in latlon_nearidx_unique])
latlon_list = list(tuple(zip(list(lat_list), list(lon_list))))

# Pickle unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list, f)
    
main_glac_rgi_wobs

unique lat/lons: 444 




,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,...,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float,CenLon_360,latlon_nearidx,latlon_unique_no,mb_fn
0,114,RGI60-05.00115,-52.1262,65.6000,5,1,72.377,143,1139,825,...,0,9,20000823,115,05.00115,5.00115,307.8738,"(98, 1231)",408,5.00115_mb_bins.csv
1,180,RGI60-05.00181,-52.9288,65.7104,5,1,2.060,111,1015,640,...,0,9,20000823,181,05.00181,5.00181,307.0712,"(97, 1228)",400,5.00181_mb_bins.csv
2,183,RGI60-05.00184,-52.9527,65.6998,5,1,2.807,241,925,625,...,0,9,20000814,184,05.00184,5.00184,307.0473,"(97, 1228)",400,5.00184_mb_bins.csv
3,257,RGI60-05.00258,-52.5941,65.8403,5,1,4.232,378,1526,1010,...,0,9,20000823,258,05.00258,5.00258,307.4059,"(97, 1230)",401,5.00258_mb_bins.csv
4,269,RGI60-05.00270,-52.5381,65.8321,5,1,5.630,351,1663,1062,...,0,9,20000823,270,05.00270,5.00270,307.4619,"(97, 1230)",401,5.00270_mb_bins.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,20158,RGI60-05.20173,-21.0378,74.7637,5,1,3.393,827,1292,1062,...,0,9,20010820,20173,05.20173,5.20173,338.9622,"(61, 1356)",249,5.20173_mb_bins.csv
607,20168,RGI60-05.20183,-22.6020,74.9837,5,1,3.712,1223,1402,1294,...,0,9,20010820,20183,05.20183,5.20183,337.3980,"(60, 1350)",245,5.20183_mb_bins.csv
608,20183,RGI60-05.20198,-22.0165,74.9015,5,1,45.809,923,1649,1297,...,0,9,20010820,20198,05.20198,5.20198,337.9835,"(60, 1352)",247,5.20198_mb_bins.csv
609,20216,RGI60-05.20231,-22.1987,75.0658,5,1,4.590,873,1465,1291,...,0,9,20010820,20231,05.20231,5.20231,337.8013,"(60, 1351)",246,5.20231_mb_bins.csv


In [6]:
# print('DELETE ME - HACK FOR DEVELOPMENT')
# print(np.where(main_glac_rgi_wobs['latlon_unique_no'] == 172)[0])
# main_glac_rgi_wobs = main_glac_rgi_wobs.loc[372:373,:]
# main_glac_rgi_wobs['mb_fn'].values

In [7]:
# ===== DEBRIS ELEVATION STATS ====================================================================================
# CALCULATE DEBRIS ELEVATION STATS FOR GLACIERS WITH DATA FOR EACH UNIQUE LAT/LON
elev_stats_latlon_dict = {}
latlon_list_updated = []
rgiid_4cal = []

for nlatlon, latlon_unique in enumerate(np.unique(main_glac_rgi_wobs.latlon_unique_no)):
# for nlatlon, latlon_unique in enumerate([np.unique(main_glac_rgi_wobs.latlon_unique_no)[0]]):

    main_glac_rgi_subset = main_glac_rgi_wobs[main_glac_rgi_wobs['latlon_unique_no'] == latlon_unique]
    main_glac_rgi_subset.reset_index(inplace=True, drop=True)
    
    # Debris elevation stats should be done by lat/lon
    df_all = None
    elev_list_all = []
    df_idx_count = 0
    count_width_passes = 0
    for nglac, glac_fn in enumerate(main_glac_rgi_subset.mb_fn.values):
#     for nglac, glac_fn in enumerate([main_glac_rgi_subset.mb_fn.values[0]]):

        glac_fullfn = debris_prms.mb_binned_fp + glac_fn
        
        glac_str_noleadzero = glac_fullfn.split('/')[-1].split('_')[0]
        rgiid = 'RGI60-' + glac_str_noleadzero.split('.')[0].zfill(2) + '.' + glac_str_noleadzero.split('.')[1]

        assert main_glac_rgi_subset.loc[nglac,'RGIId'] == rgiid, 'RGIId does not matach mass balance filename'
        
        # Select bins that meet calibratioin criteria
        df_raw = pd.read_csv(glac_fullfn)
        df = df_raw.dropna(subset=['mb_bin_mean_mwea'])
        df['z1_bin_areas_perc_cum'] = np.cumsum(df['z1_bin_area_valid_km2']) /df['z1_bin_area_valid_km2'].sum() * 100
        # add width to bins
        widths_fp = debris_prms.oggm_fp + 'widths/' + 'RGI60-' + rgiid.split('-')[1].split('.')[0] + '/'
        widths_fn = rgiid + '_widths_m.csv'
        try:
            # Add width to each elevation bin
            widths_df = pd.read_csv(widths_fp + widths_fn)
            elev_nearidx = (np.abs(df['bin_center_elev_m'].values[:,np.newaxis] - 
                                   widths_df['elev'].values).argmin(axis=1))
            df['width_m'] = widths_df.loc[elev_nearidx,'width_m'].values
        except:
            df['width_m'] = 0
        
        df_idx = np.where((df['vm_med'] <= debris_prms.vel_threshold) 
                          & (df['width_m'] >= debris_prms.width_min_dict[debris_prms.roi])
                          & (df['dc_bin_area_perc'] >= debris_prms.debrisperc_threshold)
                          & (df['dc_bin_count_valid'] >= 10)
                          & (df['z1_bin_areas_perc_cum'] <= debris_prms.term_area_perc)
                          )[0]
        df_debris = df.loc[df_idx,:]
        df_debris.reset_index(inplace=True, drop=True)
        df_idx_count += len(df_idx)
        
            
        if len(df_idx) > 0:
            for nelev, elev in enumerate(list(df_debris['bin_center_elev_m'].values)):
                elev_list_single = list(np.repeat(elev, df_debris.loc[nelev,'dc_bin_count_valid']))
                elev_list_all.extend(elev_list_single)
            
#             # only work with terminus
#             df_idx_dif = list(df_idx[1:] - df_idx[:-1])
#             if np.sum(df_idx_dif) == len(df_idx)-1:
#                 df_idx_nojump = df_idx
#             else:
#                 idx_jumpinbins = df_idx_dif.index(next(filter(lambda x: x>1, df_idx_dif)))
#                 df_idx_nojump = df_idx[0:idx_jumpinbins+1]
#             df_debris_nojump = df_debris.loc[df_idx_nojump,:]
#             df_debris_nojump.reset_index(inplace=True, drop=True)
#             # Median width to ensure terminus velocities can be estimated
#             width_median = np.median(widths_m[np.where(h < df_debris_nojump['bin_center_elev_m'].max())[0]])
#             if width_median > debris_prms.width_min_dict[debris_prms.roi]:
#                 for nelev, elev in enumerate(list(df_debris_nojump['bin_center_elev_m'].values)):
#                     elev_list_single = list(np.repeat(elev, df_debris_nojump.loc[nelev,'dc_bin_count_valid']))
#                     elev_list_all.extend(elev_list_single)
#                 count_width_passes += 1
    
            rgiid_4cal.append(rgiid.split('-')[1])
        
    if df_idx_count > 0:
        dc_zmean = np.mean(elev_list_all)
        dc_zstd = np.std(elev_list_all)
        dc_zmed = malib.fast_median(elev_list_all)
        dc_zmad = malib.mad(elev_list_all)
        
        lat_deg = float(ds.latitude[latlon_unique_dict_reversed[latlon_unique][0]].values)
        lon_deg = float(ds.longitude[latlon_unique_dict_reversed[latlon_unique][1]].values)
        elev_stats_latlon_dict[lat_deg,lon_deg] = [dc_zmean, dc_zstd, dc_zmed, dc_zmad]
        latlon_list_updated.append((lat_deg, lon_deg))
        
print('unique lat/lons updated:', len(latlon_list_updated))
# Update pickle of unique lat/lons that will be used for melt model
with open(debris_prms.latlon_unique_fp + debris_prms.latlon_unique_dict[debris_prms.roi], 'wb') as f:
    pickle.dump(latlon_list_updated, f)

unique lat/lons updated: 226


In [8]:
# Statistics of data coverage
rgiid_4cal = sorted(rgiid_4cal)
main_glac_rgi_4cal = debris_prms.selectglaciersrgitable(rgiid_4cal)
dc_area_dict = dict(zip(dc_shp.RGIId.values, dc_shp.DC_Area_v2.values))
main_glac_rgi_4cal['DC_Area_v2'] = main_glac_rgi_4cal.RGIId.map(dc_area_dict)
print('\nDC glaciers (used for cal):', main_glac_rgi_4cal.shape[0], 
      'DC Area (used for cal, km2):', main_glac_rgi_4cal.DC_Area_v2.sum() / 1e6)

321 glaciers in region 5 are included in this model run: ['00181', '00184', '00258', '00270', '00275', '00318', '00319', '00323', '00334', '00335', '00336', '00342', '00459', '00460', '00789', '00793', '00801', '00839', '00859', '00879', '01355', '01403', '01441', '01456', '01706', '02398', '02521', '02536', '03019', '03185', '03396', '03653', '03731', '03904', '04014', '04178', '04266', '04285', '04294', '04339', '04342', '04389', '04392', '04525', '04554', '04557', '04565', '04591', '04700', '04908'] and more
This study is focusing on 321 glaciers in region [5]

DC glaciers (used for cal): 321 DC Area (used for cal, km2): 727.135642


In [9]:
# ===== ADD DEBRIS ELEVATION STATS TO MET DATA ======
overwrite_dc_stats = True
for nlatlon, latlon in enumerate(latlon_list_updated):
# for nlatlon, latlon in enumerate([latlon_list_updated[0]]):
    
    lat_deg = latlon[0]
    lon_deg = latlon[1]
    
    print(nlatlon, lat_deg, lon_deg)
    
    if lat_deg < 0:
        lat_str = 'S-'
    else:
        lat_str = 'N-' 

    # ===== Meteorological data =====
    metdata_fn = debris_prms.metdata_fn_sample.replace(
        'XXXX', str(int(np.abs(lat_deg)*100)) + lat_str + str(int(lon_deg*100)) + 'E-')
    
    ds = xr.open_dataset(debris_prms.metdata_fp + metdata_fn) 
    try:
        print('  existed:', ds.dc_zmean.values, 'vs', elev_stats_latlon_dict[latlon][0])
    except:
        pass
    if 'dc_zmean' not in list(ds.keys()) or overwrite_dc_stats:
        # Add stats
        ds['dc_zmean'] = elev_stats_latlon_dict[latlon][0]
        ds['dc_zmean'].attrs = {'units':'m a.s.l.', 'long_name':'Mean debris cover elevation', 
                                'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zstd'] = elev_stats_latlon_dict[latlon][1]
        ds['dc_zstd'].attrs = {'units':'m a.s.l.', 'long_name':'Standard deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmed'] = elev_stats_latlon_dict[latlon][2]
        ds['dc_zmed'].attrs = {'units':'m a.s.l.', 'long_name':'Median debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}
        ds['dc_zmad'] = elev_stats_latlon_dict[latlon][3]
        ds['dc_zmad'].attrs = {'units':'m a.s.l.', 'long_name':'Median absolute deviation of debris cover elevation', 
                               'comment':'converted from debris cover with data that will be used for subdebris melt inversion'}

        try:
            ds.close()
        except:
            continue
            
        # Export updated dataset
        ds.to_netcdf(debris_prms.metdata_fp + metdata_fn, mode='a')
    else:
        print(lat_deg, lon_deg, 'exists')

0 82.75 316.0
1 82.75 316.75
2 82.75 319.25
3 82.75 320.5
4 82.75 325.25
5 82.75 326.75
6 82.75 327.0
7 82.75 327.25
8 82.75 327.5
9 82.75 328.25
10 82.75 328.5
11 82.75 331.0
12 82.75 331.25
13 82.75 331.5
14 82.75 335.5
15 82.75 336.0
16 82.75 336.5
17 82.75 337.0
18 82.5 315.0
19 82.5 318.75
20 82.5 321.0
21 82.25 306.0
22 82.25 310.0
23 82.25 320.0
24 82.25 324.0
25 82.0 305.0
26 82.0 309.25
27 82.0 328.25
28 81.75 302.5
29 81.75 302.75
30 81.75 303.75
31 81.75 304.0
32 81.75 304.25
33 81.75 304.5
34 81.75 305.0
35 81.75 305.25
36 81.75 309.25
37 81.75 310.25
38 81.75 310.75
39 81.75 311.25
40 81.75 326.5
41 81.75 327.0
42 81.75 329.5
43 81.75 331.25
44 81.75 334.25
45 81.5 304.5
46 81.5 304.75
47 81.5 306.75
48 81.5 308.5
49 81.5 326.25
50 81.5 330.0
51 81.5 336.25
52 81.5 337.5
53 81.5 338.0
54 81.25 302.5
55 81.0 332.75
56 81.0 338.75
57 80.75 297.5
58 80.75 335.25
59 80.75 344.75
60 80.75 345.0
61 80.5 294.25
62 80.5 296.5
63 80.5 338.5
64 80.25 294.0
65 80.25 294.25
66 80.25 2

In [11]:
print('DONE!')

DONE!
